In [1]:
import cv2
import time
import math as m
import mediapipe as mp

In [2]:
# Initilize medipipe selfie segmentation class.
mp_pose = mp.solutions.pose
mp_holistic = mp.solutions.holistic

In [3]:
def findDistance(x1, y1, x2, y2):
    dist = m.sqrt((x2-x1)**2+(y2-y1)**2)
    return dist

In [4]:
# Calculate angle.
def findAngle(x1, y1, x2, y2):
    theta = m.acos((y2 -y1)*(-y1) / (m.sqrt((x2 - x1)**2 + (y2 - y1)**2) * y1))
    degree = int(180/m.pi)*theta
    return degree

In [5]:
def sendWarning(x):
    pass

In [6]:
# Initilize frame counters.
good_frames = 0
bad_frames = 0

# Font type.
font = cv2.FONT_HERSHEY_SIMPLEX

# Colors.
blue = (255, 127, 0)
red = (50, 50, 255)
green = (127, 255, 0)
dark_blue = (127, 20, 0)
light_green = (127, 233, 100)
yellow = (0, 255, 255)
pink = (255, 0, 255)

In [ ]:
# Initialize mediapipe pose class.
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Initialize video capture object.
# For inbuilt webcam input replace file name with 0. For external camera make it 1. 
file_name = 'input.mp4'
video = cv2.VideoCapture(1)

# Meta.
fps = int(video.get(cv2.CAP_PROP_FPS))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_size = (width, height)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Initialize video writer.
video_output = cv2.VideoWriter('output.mp4', fourcc, fps, frame_size)

In [8]:
print('Processing..')
while video.isOpened():
    success, frame = video.read()
    if not success:
        print("Null.Frames")
        break
    
    # Get fps
    fps = video.get(cv2.CAP_PROP_FPS)
    # Get height and width
    h, w = frame.shape[:2]

    # Convert BGR to RGB
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the image
    keypoints = pose.process(frame)

    # Convert back to BGR
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    # Get pose landmarks
    lm = keypoints.pose_landmarks
    lmPose = mp_pose.PoseLandmark

    if lm:  # Ensure landmarks exist before accessing them
        # Extract keypoints
        l_shldr_x = int(lm.landmark[lmPose.LEFT_SHOULDER].x * w)
        l_shldr_y = int(lm.landmark[lmPose.LEFT_SHOULDER].y * h)
        r_shldr_x = int(lm.landmark[lmPose.RIGHT_SHOULDER].x * w)
        r_shldr_y = int(lm.landmark[lmPose.RIGHT_SHOULDER].y * h)
        l_ear_x = int(lm.landmark[lmPose.LEFT_EAR].x * w)
        l_ear_y = int(lm.landmark[lmPose.LEFT_EAR].y * h)
        l_hip_x = int(lm.landmark[lmPose.LEFT_HIP].x * w)
        l_hip_y = int(lm.landmark[lmPose.LEFT_HIP].y * h)

        # Calculate distances & angles
        offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        neck_inclination = findAngle(l_shldr_x, l_shldr_y, l_ear_x, l_ear_y)
        torso_inclination = findAngle(l_hip_x, l_hip_y, l_shldr_x, l_shldr_y)

        # Draw landmarks
        cv2.circle(frame, (l_shldr_x, l_shldr_y), 7, yellow, -1)
        cv2.circle(frame, (l_ear_x, l_ear_y), 7, yellow, -1)
        cv2.circle(frame, (l_shldr_x, l_shldr_y - 100), 7, yellow, -1)
        cv2.circle(frame, (r_shldr_x, r_shldr_y), 7, pink, -1)
        cv2.circle(frame, (l_hip_x, l_hip_y), 7, yellow, -1)
        cv2.circle(frame, (l_hip_x, l_hip_y - 100), 7, yellow, -1)

        # Posture assessment
        if neck_inclination < 40 and torso_inclination < 10:
            bad_frames = 0
            good_frames += 1
            cv2.putText(frame, "Good Posture", (10, 30), font, 0.9, light_green, 2)
        else:
            good_frames = 0
            bad_frames += 1
            cv2.putText(frame, "Bad Posture", (10, 30), font, 0.9, red, 2)

        # Timer display
        good_time = (1 / fps) * good_frames
        bad_time = (1 / fps) * bad_frames
        if bad_time > 180:
            sendWarning()

    # **Display the frame**
    cv2.imshow('Posture Detection', frame)

    # **Break the loop when 'q' is pressed**
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Write processed frame to output
    video_output.write(frame)

print('Finished.')
video.release()
video_output.release()
cv2.destroyAllWindows()  # Ensure all OpenCV windows close properly


Processing..


ZeroDivisionError: float division by zero